# Last FM hometask <br>
https://www.kaggle.com/ravichaubey1506/lastfm <br>
1. Выбрать данные по странам своей группы (совместно): <br>
    3530203_70101: Germany, Netherlands <br>
    3530203_70102: Belarus, Ukraine, Poland, Russian Federation<br>
    3530903_70301: Sweden, Finland, Norway, Denmark, Iceland<br>
    3530903_70302: Spain, Portugal, France, Italy, Belgium<br>
    
2. Попытаться найти полезные с точки зрения продвижения групп (или еще чего-нибудь) и нетривиальные правила, используя алгоритмы Apriori, FPGrowth, FPMax и всевозможные метрики. Хотя бы 5 правил.
3. Вывести эти правила в отдельных ячейках. 
4. Подумать, как можно было бы использовать полученные правила.

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv("lastfm.csv")
data.head()

,user,artist,sex,country
0,1,red hot chili peppers,f,Germany
1,1,the black dahlia murder,f,Germany
2,1,goldfrapp,f,Germany
3,1,dropkick murphys,f,Germany
4,1,le tigre,f,Germany


In [3]:
data.info()
data.describe()
#нет пустых значений

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 289955 entries, 0 to 289954
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   user     289955 non-null  int64 
 1   artist   289955 non-null  object
 2   sex      289955 non-null  object
 3   country  289955 non-null  object
dtypes: int64(1), object(3)
memory usage: 5.5+ MB


,user
count,289955.000000
mean,9852.460447
std,5692.355041
min,1.000000
25%,4935.000000
50%,9838.000000
75%,14769.000000
max,19718.000000


In [4]:
countries = ['Spain', 'Portugal', 'France', 'Italy', 'Belgium']

In [5]:
#оставили в датасете нужные страны
data = data.loc[data['country'].isin(countries)]
data

,user,artist,sex,country
83,6,lily allen,m,Portugal
84,6,kanye west,m,Portugal
85,6,sigur rós,m,Portugal
86,6,pink floyd,m,Portugal
87,6,stevie wonder,m,Portugal
...,...,...,...,...
289758,19707,the decemberists,m,Spain
289759,19707,my bloody valentine,m,Spain
289760,19707,bloc party,m,Spain
289761,19707,new order,m,Spain


In [6]:
#удалим строки с артистом '[unknown]'
data = data.drop(data[data['artist']=='[unknown]'].index)
data

,user,artist,sex,country
83,6,lily allen,m,Portugal
84,6,kanye west,m,Portugal
85,6,sigur rós,m,Portugal
86,6,pink floyd,m,Portugal
87,6,stevie wonder,m,Portugal
...,...,...,...,...
289758,19707,the decemberists,m,Spain
289759,19707,my bloody valentine,m,Spain
289760,19707,bloc party,m,Spain
289761,19707,new order,m,Spain


In [7]:
data_by_invoice = data.groupby(['user', 'country'])['artist'].apply(lambda x: ','.join(x)).reset_index()
data_by_invoice

,user,country,artist
0,6,Portugal,"lily allen,kanye west,sigur rós,pink floyd,ste..."
1,12,Italy,"queen,the beatles,lynyrd skynyrd,neil young,th..."
2,36,Italy,"m.i.a.,fugazi,johnny cash,clint mansell,elton ..."
3,47,Italy,"pj harvey,radiohead,zero 7,einstürzende neubau..."
4,74,Spain,"in extremo,nine inch nails,billy talent,panic!..."
...,...,...,...
1412,19690,Italy,"pink floyd,the beatles,the white stripes,depec..."
1413,19693,Belgium,"air,simon & garfunkel,pixies,serge gainsbourg,..."
1414,19697,Belgium,"underworld,pendulum,the prodigy,sufjan stevens..."
1415,19704,Italy,"keane,pinback,coldplay,travis,calexico,doves,d..."


In [13]:
data = pd.concat([data_by_invoice.drop(['country','artist'], axis=1), 
                             pd.get_dummies(data_by_invoice['country']),
                             data_by_invoice['artist'].str.get_dummies(',')
                            ], axis = 1).set_index(['user'])
data

,Belgium,France,Italy,Portugal,Spain,...and you will know us by the trail of dead,2pac,3 doors down,30 seconds to mars,311,...,wilco,within temptation,wolfgang amadeus mozart,wu-tang clan,yann tiersen,yeah yeah yeahs,yellowcard,yo la tengo,zero 7,Édith piaf
user,,,,,,,,,,,,,,,,,,,,,
6,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
36,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
47,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
74,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19690,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19693,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19697,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Работа с правилами

In [14]:
from mlxtend.frequent_patterns import apriori, association_rules
frequent_itemsets_apriori = apriori(data, min_support=0.03, use_colnames=True)
frequent_itemsets_apriori

,support,itemsets
0,0.112915,(Belgium)
1,0.230769,(France)
2,0.214538,(Italy)
3,0.086097,(Portugal)
4,0.355681,(Spain)
...,...,...
307,0.035992,"(the beatles, the rolling stones)"
308,0.031757,"(the strokes, the beatles)"
309,0.039520,"(the killers, the strokes)"
310,0.031757,"(the killers, coldplay, Spain)"


In [16]:
from mlxtend.frequent_patterns import fpgrowth
frequent_itemsets_fpgrowth = fpgrowth(data, min_support=0.05, use_colnames=True)
frequent_itemsets_fpgrowth.head()

,support,itemsets
0,0.174312,(muse)
1,0.130558,(pink floyd)
2,0.123500,(metallica)
3,0.096683,(depeche mode)
4,0.088215,(u2)


In [17]:
from mlxtend.frequent_patterns import fpmax
frequent_itemsets_fpmax = fpmax(data, min_support=0.04, use_colnames=True)
frequent_itemsets_fpmax

,support,itemsets
0,0.040226,(rihanna)
1,0.040226,(deus)
2,0.040226,(miles davis)
3,0.040932,(koЯn)
4,0.042343,(bob marley & the wailers)
...,...,...
125,0.071277,"(coldplay, Spain)"
126,0.083275,"(coldplay, radiohead)"
127,0.060692,"(radiohead, Italy)"
128,0.062103,"(radiohead, France)"


In [18]:
rules_lift = association_rules(frequent_itemsets_apriori, metric="lift", min_threshold=0.5).sort_values(by='lift', ascending=False)
rules_lift

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
168,(franz ferdinand),(kaiser chiefs),0.095977,0.050812,0.030346,0.316176,6.222529,0.025469,1.388060
169,(kaiser chiefs),(franz ferdinand),0.050812,0.095977,0.030346,0.597222,6.222529,0.025469,2.244470
189,(kaiser chiefs),(the killers),0.050812,0.110092,0.031052,0.611111,5.550926,0.025458,2.288336
188,(the killers),(kaiser chiefs),0.110092,0.050812,0.031052,0.282051,5.550926,0.025458,1.322084
105,(interpol),(bloc party),0.079746,0.083275,0.032463,0.407080,4.888406,0.025822,1.546119
...,...,...,...,...,...,...,...,...,...
52,(Spain),(nirvana),0.355681,0.124912,0.037403,0.105159,0.841864,-0.007026,0.977926
48,(Spain),(metallica),0.355681,0.123500,0.035286,0.099206,0.803288,-0.008641,0.973030
49,(metallica),(Spain),0.123500,0.355681,0.035286,0.285714,0.803288,-0.008641,0.902047
61,(radiohead),(Spain),0.237826,0.355681,0.062809,0.264095,0.742505,-0.021782,0.875546


In [19]:
#rules_lift.head(5)
#любителям Franz Ferdinand и  The killer можно посоветовать Kaiser chiefs (т.к. KC менее популярны, чем FF и TK)
rules_lift[(rules_lift['lift'] > 5.0) & ( rules_lift['conviction'] < 2.0)].head(5)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
168,(franz ferdinand),(kaiser chiefs),0.095977,0.050812,0.030346,0.316176,6.222529,0.025469,1.388060
188,(the killers),(kaiser chiefs),0.110092,0.050812,0.031052,0.282051,5.550926,0.025458,1.322084


In [35]:
rules_lift2 = rules_lift[(rules_lift['consequents'] != frozenset('Spain'))
                         &(rules_lift['consequents'] != frozenset('France'))
                         &(rules_lift['consequents'] != frozenset('Portugal'))
                         &(rules_lift['consequents'] != frozenset('Belgium'))
                         &(rules_lift['consequents'] != frozenset('Italy'))
                        & (rules_lift['support'] < 0.1)
                        ].sort_values(by='lift', ascending=True)
rules_lift2.head(1)
#Radiohead не стоит ехать с большими концертами в Испанию, они там не особо популярны

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
60,(Spain),(radiohead),0.355681,0.237826,0.062809,0.176587,0.742505,-0.021782,0.925628
